In [37]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [38]:
X=pd.read_csv('X.csv')

a1=list(X.Region.unique())
regions_dict=dict(zip(range(1,len(a1)+1),a1))
def my_date(dt):
    a= dt.split('-')
    return datetime.date(int(a[0]),int(a[1]),int(a[2]))
X.Date=X.Date.apply(my_date)
myday = list(X.tail(1).Date)[0]

In [39]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13535 entries, 0 to 13534
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                13535 non-null  object 
 1   Confirmed           13535 non-null  int64  
 2   Recovered           13535 non-null  int64  
 3   Deaths              13535 non-null  int64  
 4   Day_deaths          13535 non-null  int64  
 5   Day_confirmed       13535 non-null  int64  
 6   Day_recovered       13535 non-null  int64  
 7   Rt                  12378 non-null  float64
 8   Rt2                 12488 non-null  float64
 9   Rate_conf           13535 non-null  float64
 10  Rate_ill            13535 non-null  float64
 11  Region              13535 non-null  object 
 12  Remaining_ill       13535 non-null  int64  
 13  MA7_dayconfirmed    13019 non-null  float64
 14  MA14_dayconfirmed   12417 non-null  float64
 15  MA7_remaining_ill   13019 non-null  float64
 16  MA14

In [40]:
covid_rates = pd.DataFrame()
mask = (X.Date==myday)&(X.Region!='Россия')
today_cases = X[mask]
#covid_rates.reset_index(drop=True)

covid_rates['По количеству текущих больных'] = today_cases.sort_values(by=['Remaining_ill'],ascending = False).Region.reset_index(drop=True)
covid_rates['По количеству новых'] = today_cases.sort_values(by=['Day_confirmed'],ascending = False).Region.reset_index(drop=True)
covid_rates['По умершим за день'] = today_cases.sort_values(by=['Day_deaths'],ascending = False).Region.reset_index(drop=True)
covid_rates['По выздоровевшим за день'] = today_cases.sort_values(by=['Day_recovered'],ascending = False).Region.reset_index(drop=True)

fig =  ff.create_table(covid_rates.head(15))
fig.update_layout(
    
                  title_text ='Таблица рейтингов заболеваемости COVID-19 по регионам',
                  margin = {'t':100, 'b':100},
                  title_x = 0.5,
                  title_y= 0.95,
                  title_xanchor = "center",
                  title_yanchor = "top", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)
fig.show()




In [41]:
region_number = 5
regions_towatch = list(covid_rates['По количеству текущих больных'].head(region_number))
regions_towatch.append("Россия")
regions_towatch

['Москва',
 'Московская обл.',
 'Санкт-Петербург',
 'Челябинская обл.',
 'Саратовская обл.',
 'Россия']

In [42]:
mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))
#mask = (X.Region.isin(regions_towatch))

fig = px.line(X[mask], y="Day_confirmed", x="Date",
                             color="Region", line_group="Region",
              line_shape="spline", render_mode="notebook",labels={'Region':'Регион:'},
              color_discrete_map={
                "Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия': 'aquamarine'},
             title="График количества новых больных по 5 регионам с наибольшим количеством заболеваний")

fig.update_layout(
    annotations=[
            dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper")],
           
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)

fig.show()






In [43]:
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))

colors_dict = {"Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия':'aquamarine'}
fig = go.Figure()
for i in regions_towatch:
    mask = X['Region'] == i
        
    fig.add_trace(
        
        go.Bar(
        name = f'{i}',
        x=X[mask].Date,
        y=X[mask].Rt,
        text=round(X[mask].Rt,3),
        textposition='inside',
        marker_color= colors_dict.get(i),
        
        showlegend=True))




                  
fig.update_layout(
    annotations=[
        dict(
            x='2020-04-13',
            y=1.717,
            xref="x",
            yref="y",
            text="Введение<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
        dict(
            x='2020-06-09',
            y=0.942,
            xref="x",
            yref="y",
            text="Отмена<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
            dict(x=0.5,y=-0.15,
            showarrow=False,
            text="Данные : Яндекс <br> формула расчета из методических рекомендациях Роспотребнадзора.",
            xref="paper",
            yref="paper")],
                  title = "Коэффициент распространения (Rt) по 5 регионам ,<br> с максимальным количеством новых заболеваний ",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.9,
                  width = 990, height = 600,template = 'gridon',
    
    
                 # xaxis_title='Коэффициент рассчитан, исходя из данных оперативного штаба и <br>' + 
                               #' формулы расчета в методических рекомендациях Роспотребнадзора. ',
    yaxis_title = ' '
)


fig.show()


In [44]:
print('Все регионы по которым есть статистика:')
print('*'*40)
print(regions_dict.items())
print('*'*40)
print('Рассматриваем регионы: ')
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))


my_region = regions_dict[41]
print('Выбрали регион: ',my_region)


Все регионы по которым есть статистика:
****************************************
dict_items([(1, 'Адыгея'), (2, 'Алтай'), (3, 'Алтайский край'), (4, 'Амурская обл.'), (5, 'Архангельская обл.'), (6, 'Астраханская обл.'), (7, 'Башкортостан'), (8, 'Белгородская обл.'), (9, 'Брянская обл.'), (10, 'Бурятия'), (11, 'Владимирская обл.'), (12, 'Волгоградская обл.'), (13, 'Вологодская обл.'), (14, 'Воронежская обл.'), (15, 'Дагестан'), (16, 'Еврейская АО'), (17, 'Забайкальский край'), (18, 'Ивановская обл.'), (19, 'Ингушетия'), (20, 'Иркутская обл.'), (21, 'Кабардино-Балкария'), (22, 'Калининградская обл.'), (23, 'Калмыкия'), (24, 'Калужская обл.'), (25, 'Камчатский край'), (26, 'Карачаево-Черкессия'), (27, 'Карелия'), (28, 'Кемеровская обл.'), (29, 'Кировская обл.'), (30, 'Коми'), (31, 'Костромская обл.'), (32, 'Краснодарский край'), (33, 'Красноярский край'), (34, 'Крым'), (35, 'Курганская обл.'), (36, 'Курская обл.'), (37, 'Ленинградская обл.'), (38, 'Липецкая обл.'), (39, 'Магаданская обл.'

In [45]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region.isin(regions_towatch))
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region==my_region)

fig = px.bar(X[mask], x='Date', 
             y="Day_confirmed",color ='Day_confirmed',
             title=f'{my_region}: '+'график количества новых заболевших COVID-19 по датам',
              color_continuous_scale= 'jet',text = 'Day_confirmed',
             labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)
fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask]['Date'],
        y=X[mask]['MA7_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=1355,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=1572,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-90),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper"),
        
                ],
    
    
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.15,legend_y = 0.95,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ' '
)


fig.show()




In [46]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region==my_region)
#Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
#Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = X[mask].Date,y=X[mask].Change_dayconf,text = X[mask]['Change_dayconf'],textposition = 'inside',
                
                
                 marker_color = X[mask]['Change_dayconf'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}}
                    )   
                )
fig.update_layout(
   annotations=[
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика новых заболевших COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Данные: Яндекс',
    yaxis_title = ' '
)

fig.show()






In [47]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region.isin(regions_towatch))

fig = px.bar(X[mask], x='Date', y="Remaining_ill",color ='Remaining_ill',
             title=f'{my_region}: '+'график количества текущих больных COVID-19 по датам',
              color_continuous_scale= 'tempo',text = 'Remaining_ill',
            labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA7_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=85824,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                                '<br>Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()





In [48]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region==my_region)
Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_remill,text = Y['Change_remill'],textposition = 'inside',
                
                #marker_color=Y['Change_remill'].apply(lambda d:'crimson' if d>0 else 'green'),
                marker_color = Y['Change_remill'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}})     
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-3877,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
#         dict(
#             x='2020-05-20',
#             y=-2816,
#             xref="x",
#             yref="y",
#             text="20 мая",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                               '<br>Данные : Яндекс',
    yaxis_title = ' '
)

fig.show()


In [49]:
delta = datetime.timedelta(14)
xday=myday-delta
print(xday)

2020-08-01


In [50]:
delta = datetime.timedelta(14)
past14=myday-delta
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))

def my_round(k):
    return round(k,3)

# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)

d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.tail(10)
print(regions_towatch)

[[1.013 1.002 1.009 1.007 1.    1.01  1.002 1.004 1.005 1.002 1.007 1.007
  1.007 1.001 0.999]
 [1.041 1.099 1.128 1.126 1.103 1.07  1.039 1.021 0.98  0.967 0.964 0.95
  0.968 0.982 0.987]
 [0.967 0.982 0.989 0.981 0.966 0.961 0.959 0.976 0.987 0.987 0.98  0.973
  0.967 0.972 0.991]
 [0.932 0.949 0.964 0.982 0.985 0.981 0.98  0.977 0.991 0.994 0.998 1.013
  1.006 1.013 1.021]
 [1.013 1.005 0.987 0.967 0.962 0.956 0.969 0.987 0.997 1.024 1.024 1.021
  1.013 0.997 1.005]
 [0.929 0.896 0.893 0.889 0.9   0.895 0.878 0.861 0.854 0.867 0.897 0.943
  0.939 0.955 0.94 ]]
['Москва', 'Московская обл.', 'Санкт-Петербург', 'Челябинская обл.', 'Саратовская обл.', 'Россия']


In [51]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print('-'*40)
print(x2)

[datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15)]
----------------------------------------
['1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020']


In [52]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [53]:
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt2.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head(10)

[[1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011
  1.011 1.011 1.011]
 [1.009 1.009 1.009 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
  1.009 1.009 1.009]
 [1.027 1.027 1.026 1.026 1.026 1.025 1.025 1.025 1.024 1.024 1.024 1.024
  1.023 1.023 1.023]
 [1.022 1.021 1.021 1.021 1.02  1.02  1.02  1.02  1.02  1.02  1.019 1.019
  1.019 1.019 1.019]
 [1.043 1.043 1.042 1.042 1.041 1.04  1.039 1.039 1.038 1.038 1.038 1.038
  1.038 1.037 1.037]
 [1.046 1.044 1.043 1.041 1.039 1.038 1.036 1.035 1.033 1.032 1.03  1.03
  1.029 1.029 1.028]]


Date,Region,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15
0,Москва,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011
1,Московская обл.,1.009,1.009,1.009,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.009,1.009,1.009
2,Россия,1.027,1.027,1.026,1.026,1.026,1.025,1.025,1.025,1.024,1.024,1.024,1.024,1.023,1.023,1.023
3,Санкт-Петербург,1.022,1.021,1.021,1.021,1.020,1.020,1.020,1.020,1.020,1.020,1.019,1.019,1.019,1.019,1.019
4,Саратовская обл.,1.043,1.043,1.042,1.042,1.041,1.040,1.039,1.039,1.038,1.038,1.038,1.038,1.038,1.037,1.037
5,Челябинская обл.,1.046,1.044,1.043,1.041,1.039,1.038,1.036,1.035,1.033,1.032,1.030,1.030,1.029,1.029,1.028


In [54]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print(x2)

[datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15)]
['1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020']


In [55]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt2 по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [56]:
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))

Z = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values
Z.head()

Date,Region,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15
0,Москва,690,664,693,691,687,684,686,691,689,694,694,689,692,688,695
1,Московская обл.,149,157,160,161,162,159,158,161,149,153,154,152,148,156,153
2,Россия,5462,5427,5394,5159,5204,5267,5241,5212,5189,5118,4945,5102,5057,5065,5061
3,Санкт-Петербург,161,163,160,158,156,159,157,155,160,157,157,161,160,159,162
4,Саратовская обл.,96,95,94,93,94,90,94,95,96,95,94,96,95,94,97


In [57]:



x1=list(Z.columns[1:])
y=list(Z.Region)
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
print(x2)


['1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020']


In [58]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,
    
    
                  xaxis_title='Динамика новых заболевших по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()

In [59]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_conf'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_conf']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(15)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6605,2020-08-15,251686,196098,4645,12,695,1275,0.999,1.011,27.420240,29.471352,Москва,50943,691.571429,688.357143,52302.571429,53795.928571,7,-592
6772,2020-08-15,65781,49381,1141,2,153,156,0.987,1.009,7.166592,8.827579,Московская обл.,15259,152.142857,155.928571,15305.142857,15403.357143,-3,-5
9457,2020-08-15,33846,24502,2245,20,162,76,1.021,1.019,3.687394,4.106887,Санкт-Петербург,7099,159.428571,158.857143,6912.142857,6649.142857,3,66
7244,2020-08-15,25635,22577,429,3,125,215,1.070,1.020,2.792837,1.520919,Нижегородская обл.,2629,123.428571,120.285714,2835.857143,3099.857143,-3,-93
10085,2020-08-15,23255,18955,367,5,158,77,0.904,1.030,2.533545,2.275304,Свердловская обл.,3933,167.428571,176.500000,3702.285714,3408.071429,4,76
12283,2020-08-15,18428,16078,148,2,103,209,0.865,1.025,2.007661,1.273893,ХМАО – Югра,2202,115.571429,137.500000,2343.571429,2630.714286,-3,-108
3136,2020-08-15,15324,11198,225,1,113,109,0.940,1.032,1.669492,2.256792,Иркутская обл.,3901,117.714286,125.428571,4020.857143,4217.071429,3,3
5181,2020-08-15,15051,12485,387,4,119,74,0.955,1.034,1.639750,1.260587,Красноярский край,2179,123.000000,125.928571,2029.714286,1772.357143,-1,41
8981,2020-08-15,14867,12301,284,6,128,184,0.992,1.036,1.619704,1.320174,Ростовская обл.,2282,126.714286,128.071429,2387.714286,2420.214286,2,-62
12754,2020-08-15,12589,8134,99,0,76,98,0.940,1.028,1.371524,2.520017,Челябинская обл.,4356,84.285714,91.571429,4399.285714,4407.857143,-4,-22


In [60]:



fig = px.pie(df2, values='Confirmed', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего выявленных случаев заболевания COVID-19 в России: {df2.Confirmed.sum()}<br>'+
            f'(на дату: {myday})'
            ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  margin = {'t':150, 'b':0},
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [61]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_ill'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_ill']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(20)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6605,2020-08-15,251686,196098,4645,12,695,1275,0.999,1.011,27.420240,29.471352,Москва,50943,691.571429,688.357143,52302.571429,53795.928571,7,-592
6772,2020-08-15,65781,49381,1141,2,153,156,0.987,1.009,7.166592,8.827579,Московская обл.,15259,152.142857,155.928571,15305.142857,15403.357143,-3,-5
9457,2020-08-15,33846,24502,2245,20,162,76,1.021,1.019,3.687394,4.106887,Санкт-Петербург,7099,159.428571,158.857143,6912.142857,6649.142857,3,66
12754,2020-08-15,12589,8134,99,0,76,98,0.940,1.028,1.371524,2.520017,Челябинская обл.,4356,84.285714,91.571429,4399.285714,4407.857143,-4,-22
9614,2020-08-15,10824,6755,66,1,97,60,1.005,1.037,1.179234,2.315800,Саратовская обл.,4003,95.285714,94.428571,3956.285714,3869.857143,3,36
10085,2020-08-15,23255,18955,367,5,158,77,0.904,1.030,2.533545,2.275304,Свердловская обл.,3933,167.428571,176.500000,3702.285714,3408.071429,4,76
3136,2020-08-15,15324,11198,225,1,113,109,0.940,1.032,1.669492,2.256792,Иркутская обл.,3901,117.714286,125.428571,4020.857143,4217.071429,3,3
1880,2020-08-15,10576,6834,78,1,96,97,1.000,1.038,1.152215,2.119683,Волгоградская обл.,3664,97.714286,95.642857,3641.571429,3627.857143,-1,-2
10713,2020-08-15,9956,6789,193,4,127,69,0.914,1.048,1.084669,1.720507,Ставропольский край,2974,117.142857,115.357143,2867.142857,2756.071429,-3,54
6929,2020-08-15,11787,8710,112,3,67,137,0.709,1.026,1.284149,1.715301,Мурманская обл.,2965,76.428571,101.214286,3177.571429,3409.214286,3,-73


In [62]:


fig = px.pie(df2, values='Remaining_ill', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
            f'(на дату: {myday})'
             ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':150, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [63]:
fig_r8 = go.Figure(data=[go.Pie(labels=df2.Region, values=df2.Remaining_ill, textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig_r8.update_layout(
                title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
                f'(на дату: {myday})',
                 margin = {'t':120, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  width = 900, height=900,
                  title_xanchor = "center",
                  title_yanchor = "bottom")
fig_r8.show()

In [64]:
delta2 = datetime.timedelta(7)
past7=myday-delta2
mask = (X.Date>=past7)&(X.Region.isin(regions_towatch))&(X.Region!='Россия')
                                      
box_cases1 = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum()
box_cases2 = X[mask].groupby(['Region','Date'])['Day_recovered'].sum()
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))

42 Москва
43 Московская обл.
56 Россия
60 Санкт-Петербург
61 Саратовская обл.
81 Челябинская обл.


In [65]:
new_df1=box_cases1.unstack().T
new_df1.tail(20)

Region,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Челябинская обл.
Date,,,,,
2020-08-08,691,161,155,95,87
2020-08-09,689,149,160,96,94
2020-08-10,694,153,157,95,83
2020-08-11,694,154,157,94,85
2020-08-12,689,152,161,96,88
2020-08-13,692,148,160,95,84
2020-08-14,688,156,159,94,80
2020-08-15,695,153,162,97,76


In [66]:

cols = list(new_df1.columns)
color_dict=dict(zip(cols,['green','darkblue','goldenrod','magenta','red']))
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по новым заболевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Новые заболевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [67]:
new_df2=box_cases2.unstack().T
new_df2.tail(20)

Region,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Челябинская обл.
Date,,,,,
2020-08-08,1278,191,83,86,119
2020-08-09,803,176,33,20,29
2020-08-10,898,169,45,41,78
2020-08-11,1043,160,78,62,8
2020-08-12,1088,215,93,120,128
2020-08-13,787,150,96,93,102
2020-08-14,1350,159,72,81,105
2020-08-15,1275,156,76,60,98


In [68]:

cols = list(new_df2.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по выздоровевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Выздоровевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [69]:
from plotly.subplots import make_subplots

# fig = tools.make_subplots(rows = 1, cols = 2, specs = [[{}, {}]],
#                           horizontal_spacing = 0.05)


fig = make_subplots(rows=2, cols=1,specs = [[{}],[{}]],vertical_spacing = 0.04,shared_xaxes=True,
                  subplot_titles=("Box-график распределения по новым заболевшим(в день) за неделю:",
                                 "Box-график распределения по выздоровевшим(в день) за неделю:"))
#subplot_titles=("Статистика по новым заболевшим за последние 2 недели","Статистика по выздоровевшим за последние 2 недели")

for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)),row=1,col=1)

for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)),row=2,col=1)


fig.update_layout(
#     annotations=[
#         dict(
#             x=0.5,
#             y=-0.15,
#             showarrow=False,
#             text="Ветикальная пунктирная линия - среднее, Вертикальная сплошная - медиана распределения",
#             xref="paper",
#             yref="paper"),
        
#                 ],
    
    
                  width = 990, height = 1200,
    
    
                 showlegend=False,template ='gridon'
)

#fig.update_layout(height=1000, width=800,showlegend=False)
fig.show()